# Conversation Service

In [1]:
import json
import sys
import os
sys.path.append(os.path.join(os.getcwd(),'..','..'))
import watson_developer_cloud

In [2]:
USERNAME = os.environ.get('CONVERSATION_USERNAME','<USERNAME>')
PASSWORD = os.environ.get('CONVERSATION_PASSWORD','<PASSWORD>')

In [4]:
conversation = watson_developer_cloud.ConversationV1(username=USERNAME,
                                                     password=PASSWORD,
                                                     version='2017-04-21')

## Pizza Chatbot

Let's create and chat with a simple pizza bot. We'll start by defining the bot's workspace, then add intents and examples to recognize pizza orders. Once the chatbot is configured, we'll send a message to converse with our pizza bot.

In [5]:
# define the dialog for our example workspace
dialog_nodes = [{'conditions': '#order_pizza',
                 'context': None,
                 'description': None,
                 'dialog_node': 'YesYouCan',
                 'go_to': None,
                 'metadata': None,
                 'output': {'text': {'selection_policy': 'random',
                                     'values': ['Yes You can!', 'Of course!']}},
                 'parent': None,
                 'previous_sibling': None,}]

# create an example workspace
workspace = conversation.create_workspace(name='example_workspace',
                                          description='An example workspace.',
                                          language='en',
                                          dialog_nodes=dialog_nodes)

# print response and save workspace_id
print(json.dumps(workspace, indent=2))
workspace_id=workspace['workspace_id']

{
  "name": "example_workspace",
  "created": "2017-05-04T19:44:35.869Z",
  "updated": "2017-05-04T19:44:35.869Z",
  "language": "en",
  "metadata": null,
  "description": "An example workspace.",
  "workspace_id": "e326ef7b-bde6-4c6c-9573-aedd9e5a70e8"
}


In [6]:
# add an intent to the workspace
intent = conversation.create_intent(workspace_id=workspace_id,
                                    intent='order_pizza',
                                    description='A pizza order.')
print(json.dumps(intent, indent=2))

{
  "intent": "order_pizza",
  "created": "2017-05-04T19:44:36.371Z",
  "updated": "2017-05-04T19:44:36.371Z",
  "description": "A pizza order."
}


In [7]:
# add examples to the intent
example1 = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='Can I order a pizza?')
example2 = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='I want to order a pizza.')
example3 = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='pizza order')
example4 = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='pizza to go')

print(json.dumps(example1, indent=2))
print(json.dumps(example2, indent=2))
print(json.dumps(example3, indent=2))
print(json.dumps(example4, indent=2))

{
  "text": "Can I order a pizza?",
  "created": "2017-05-04T19:44:36.915Z",
  "updated": "2017-05-04T19:44:36.915Z"
}
{
  "text": "I want to order a pizza.",
  "created": "2017-05-04T19:44:37.393Z",
  "updated": "2017-05-04T19:44:37.393Z"
}
{
  "text": "pizza order",
  "created": "2017-05-04T19:44:37.932Z",
  "updated": "2017-05-04T19:44:37.932Z"
}
{
  "text": "pizza to go",
  "created": "2017-05-04T19:44:38.404Z",
  "updated": "2017-05-04T19:44:38.404Z"
}


In [8]:
# check workspace status (wait for training to complete)
workspace = conversation.get_workspace(workspace_id=workspace_id)
print('The workspace status is: {0}'.format(workspace['status']))
if workspace['status'] == 'Available':
    print('Ready to chat!')
else:
    print('The workspace should be available shortly. Please try again in 30s.')
    print('(You can send messages, but not all functionality will be supported yet.)')

The workspace status is: Training
The workspace should be available shortly. Please try again in 30s.
(You can send messages, but not all functionality will be supported yet.)


In [9]:
# start a chat with the pizza bot
input = {'text': 'Can I order a pizza?'}
response = conversation.message(workspace_id=workspace_id,
                                input=input)
print(json.dumps(response, indent=2))

{
  "intents": [],
  "entities": [],
  "input": {
    "text": "Can I order a pizza?"
  },
  "output": {
    "log_messages": [
      {
        "level": "warn",
        "msg": "No dialog node matched for the input at a root level."
      },
      {
        "level": "warn",
        "msg": "No dialog node condition matched to true in the last dialog round - context.nodes_visited is empty. Falling back to the root node in the next round."
      }
    ],
    "text": []
  },
  "context": {
    "conversation_id": "2fa01d07-f0cb-4b35-b158-d4448016bb5b",
    "system": {
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ],
      "dialog_turn_counter": 1,
      "dialog_request_counter": 1
    }
  }
}


In [10]:
# continue a chat with the pizza bot
# (when you send multiple requests for the same conversation,
#  then include the context object from the previous response)
input = {'text': 'medium'}
response = conversation.message(workspace_id=workspace_id,
                                input=input,
                                context=response['context'])
print(json.dumps(response, indent=2))

{
  "intents": [],
  "entities": [],
  "input": {
    "text": "medium"
  },
  "output": {
    "log_messages": [
      {
        "level": "warn",
        "msg": "No dialog node matched for the input at a root level."
      },
      {
        "level": "warn",
        "msg": "No dialog node condition matched to true in the last dialog round - context.nodes_visited is empty. Falling back to the root node in the next round."
      }
    ],
    "text": []
  },
  "context": {
    "conversation_id": "2fa01d07-f0cb-4b35-b158-d4448016bb5b",
    "system": {
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ],
      "dialog_turn_counter": 2,
      "dialog_request_counter": 2,
      "branch_exited_reason": "fallback"
    }
  }
}


## Operation Examples

The following examples demonstrate each operation of the Conversation service. They use the pizza chatbot workspace configured above.

### Message

In [11]:
input = {'text': 'Can I order a pizza?'}
response = conversation.message(workspace_id=workspace_id,
                                input=input)
print(json.dumps(response, indent=2))

{
  "intents": [],
  "entities": [],
  "input": {
    "text": "Can I order a pizza?"
  },
  "output": {
    "log_messages": [
      {
        "level": "warn",
        "msg": "No dialog node matched for the input at a root level."
      },
      {
        "level": "warn",
        "msg": "No dialog node condition matched to true in the last dialog round - context.nodes_visited is empty. Falling back to the root node in the next round."
      }
    ],
    "text": []
  },
  "context": {
    "conversation_id": "19de6478-695a-4440-8d76-eaa82b22ea69",
    "system": {
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ],
      "dialog_turn_counter": 1,
      "dialog_request_counter": 1
    }
  }
}


### Workspaces

In [12]:
response = conversation.create_workspace(name='test_workspace',
                                         description='Test workspace.',
                                         language='en',
                                         metadata={})
print(json.dumps(response, indent=2))
test_workspace_id = response['workspace_id']

{
  "name": "test_workspace",
  "created": "2017-05-04T19:44:41.152Z",
  "updated": "2017-05-04T19:44:41.152Z",
  "language": "en",
  "metadata": {},
  "description": "Test workspace.",
  "workspace_id": "f21667ef-ba51-4e20-9f46-784970d7b663"
}


In [13]:
response = conversation.delete_workspace(workspace_id=test_workspace_id)
print(json.dumps(response, indent=2))

{}


In [14]:
response = conversation.get_workspace(workspace_id=workspace_id, export=True)
print(json.dumps(response, indent=2))

{
  "name": "example_workspace",
  "created": "2017-05-04T19:44:35.869Z",
  "intents": [
    {
      "intent": "order_pizza",
      "created": "2017-05-04T19:44:36.371Z",
      "updated": "2017-05-04T19:44:38.404Z",
      "examples": [
        {
          "text": "Can I order a pizza?",
          "created": "2017-05-04T19:44:36.915Z",
          "updated": "2017-05-04T19:44:36.915Z"
        },
        {
          "text": "I want to order a pizza.",
          "created": "2017-05-04T19:44:37.393Z",
          "updated": "2017-05-04T19:44:37.393Z"
        },
        {
          "text": "pizza order",
          "created": "2017-05-04T19:44:37.932Z",
          "updated": "2017-05-04T19:44:37.932Z"
        },
        {
          "text": "pizza to go",
          "created": "2017-05-04T19:44:38.404Z",
          "updated": "2017-05-04T19:44:38.404Z"
        }
      ],
      "description": "A pizza order."
    }
  ],
  "updated": "2017-05-04T19:44:38.404Z",
  "entities": [],
  "language": "en",
  

In [15]:
response = conversation.list_workspaces()
print(json.dumps(response, indent=2))

{
  "workspaces": [
    {
      "name": "Car_Dashboard",
      "created": "2016-07-19T16:31:17.236Z",
      "updated": "2017-05-04T17:11:53.494Z",
      "language": "en",
      "metadata": {
        "runtime_version": "2016-09-20"
      },
      "description": "Cognitive Car workspace which allows multi-turn conversations to perform tasks in the car.",
      "workspace_id": "8d869397-411b-4f0a-864d-a2ba419bb249"
    },
    {
      "name": "example_workspace",
      "created": "2017-05-04T19:37:05.567Z",
      "updated": "2017-05-04T19:37:23.463Z",
      "language": "en",
      "metadata": null,
      "description": "An example workspace for ordering pizza.",
      "workspace_id": "745166ef-ff47-4a02-888e-eb145fbc22dd"
    },
    {
      "name": "example_workspace",
      "created": "2017-05-04T19:44:09.895Z",
      "updated": "2017-05-04T19:44:17.600Z",
      "language": "en",
      "metadata": null,
      "description": "An example workspace for ordering pizza.",
      "workspace_id":

In [16]:
response = conversation.update_workspace(workspace_id=workspace_id,
                                         description='An example workspace for ordering pizza.')
print(json.dumps(response, indent=2))

{
  "name": "example_workspace",
  "created": "2017-05-04T19:44:35.869Z",
  "updated": "2017-05-04T19:44:43.174Z",
  "language": "en",
  "metadata": null,
  "description": "An example workspace for ordering pizza.",
  "workspace_id": "e326ef7b-bde6-4c6c-9573-aedd9e5a70e8"
}


### Intents

In [17]:
response = conversation.create_intent(workspace_id=workspace_id,
                                      intent='test_intent',
                                      description='Test intent.')
print(json.dumps(response, indent=2))

{
  "intent": "test_intent",
  "created": "2017-05-04T19:44:43.723Z",
  "updated": "2017-05-04T19:44:43.723Z",
  "description": "Test intent."
}


In [18]:
response = conversation.delete_intent(workspace_id=workspace_id,
                                      intent='test_intent')
print(json.dumps(response, indent=2))

{}


In [19]:
response = conversation.get_intent(workspace_id=workspace_id,
                                   intent='order_pizza',
                                   export=True)
print(json.dumps(response, indent=2))

{
  "intent": "order_pizza",
  "created": "2017-05-04T19:44:36.371Z",
  "updated": "2017-05-04T19:44:38.404Z",
  "examples": [
    {
      "text": "Can I order a pizza?",
      "created": "2017-05-04T19:44:36.915Z",
      "updated": "2017-05-04T19:44:36.915Z"
    },
    {
      "text": "I want to order a pizza.",
      "created": "2017-05-04T19:44:37.393Z",
      "updated": "2017-05-04T19:44:37.393Z"
    },
    {
      "text": "pizza order",
      "created": "2017-05-04T19:44:37.932Z",
      "updated": "2017-05-04T19:44:37.932Z"
    },
    {
      "text": "pizza to go",
      "created": "2017-05-04T19:44:38.404Z",
      "updated": "2017-05-04T19:44:38.404Z"
    }
  ],
  "description": "A pizza order."
}


In [20]:
response = conversation.list_intents(workspace_id=workspace_id,
                                     export=True)
print(json.dumps(response, indent=2))

{
  "intents": [
    {
      "intent": "order_pizza",
      "created": "2017-05-04T19:44:36.371Z",
      "updated": "2017-05-04T19:44:38.404Z",
      "examples": [
        {
          "text": "Can I order a pizza?",
          "created": "2017-05-04T19:44:36.915Z",
          "updated": "2017-05-04T19:44:36.915Z"
        },
        {
          "text": "I want to order a pizza.",
          "created": "2017-05-04T19:44:37.393Z",
          "updated": "2017-05-04T19:44:37.393Z"
        },
        {
          "text": "pizza order",
          "created": "2017-05-04T19:44:37.932Z",
          "updated": "2017-05-04T19:44:37.932Z"
        },
        {
          "text": "pizza to go",
          "created": "2017-05-04T19:44:38.404Z",
          "updated": "2017-05-04T19:44:38.404Z"
        }
      ],
      "description": "A pizza order."
    }
  ],
  "pagination": {
    "refresh_url": "/v1/workspaces/e326ef7b-bde6-4c6c-9573-aedd9e5a70e8/intents?include_count=none&version=2017-04-21&export=true"
  }


In [21]:
response = conversation.update_intent(workspace_id=workspace_id,
                                      intent='order_pizza',
                                      new_intent='order_pizza',
                                      new_description='Order a pizza.')
print(json.dumps(response, indent=2))

{
  "intent": "order_pizza",
  "created": "2017-05-04T19:44:36.371Z",
  "updated": "2017-05-04T19:44:45.616Z",
  "description": "Order a pizza."
}


### Examples

In [22]:
response = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='Gimme a pizza with pepperoni')
print(json.dumps(response, indent=2))

{
  "text": "Gimme a pizza with pepperoni",
  "created": "2017-05-04T19:44:46.237Z",
  "updated": "2017-05-04T19:44:46.237Z"
}


In [23]:
response = conversation.delete_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='Gimme a pizza with pepperoni')
print(json.dumps(response, indent=2))

{}


In [24]:
response = conversation.get_example(workspace_id=workspace_id,
                                    intent='order_pizza',
                                    text='I want to order a pizza.')
print(json.dumps(response, indent=2))

{
  "text": "I want to order a pizza.",
  "created": "2017-05-04T19:44:37.393Z",
  "updated": "2017-05-04T19:44:37.393Z"
}


In [25]:
response = conversation.list_examples(workspace_id=workspace_id,
                                      intent='order_pizza')
print(json.dumps(response, indent=2))

{
  "examples": [
    {
      "text": "Can I order a pizza?",
      "created": "2017-05-04T19:44:36.915Z",
      "updated": "2017-05-04T19:44:36.915Z"
    },
    {
      "text": "I want to order a pizza.",
      "created": "2017-05-04T19:44:37.393Z",
      "updated": "2017-05-04T19:44:37.393Z"
    },
    {
      "text": "pizza order",
      "created": "2017-05-04T19:44:37.932Z",
      "updated": "2017-05-04T19:44:37.932Z"
    },
    {
      "text": "pizza to go",
      "created": "2017-05-04T19:44:38.404Z",
      "updated": "2017-05-04T19:44:38.404Z"
    }
  ],
  "pagination": {
    "refresh_url": "/v1/workspaces/e326ef7b-bde6-4c6c-9573-aedd9e5a70e8/intents/order_pizza/examples?include_count=none&version=2017-04-21"
  }
}


In [26]:
response = conversation.update_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='I want to order a pizza.',
                                       new_text='I want to order a pizza with pepperoni.')
print(json.dumps(response, indent=2))

{
  "text": "I want to order a pizza with pepperoni.",
  "created": "2017-05-04T19:44:37.393Z",
  "updated": "2017-05-04T19:44:48.409Z"
}


### Counterexamples

In [27]:
response = conversation.create_counterexample(workspace_id=workspace_id,
                                              text='I want financial advice today.')
print(json.dumps(response, indent=2))

{
  "text": "I want financial advice today.",
  "created": "2017-05-04T19:44:48.903Z",
  "updated": "2017-05-04T19:44:48.903Z"
}


In [28]:
response = conversation.get_counterexample(workspace_id=workspace_id,
                                           text='I want financial advice today.')
print(json.dumps(response, indent=2))

{
  "text": "I want financial advice today.",
  "created": "2017-05-04T19:44:48.903Z",
  "updated": "2017-05-04T19:44:48.903Z"
}


In [29]:
response = conversation.list_counterexamples(workspace_id=workspace_id)
print(json.dumps(response, indent=2))

{
  "counterexamples": [
    {
      "text": "I want financial advice today.",
      "created": "2017-05-04T19:44:48.903Z",
      "updated": "2017-05-04T19:44:48.903Z"
    }
  ],
  "pagination": {
    "refresh_url": "/v1/workspaces/e326ef7b-bde6-4c6c-9573-aedd9e5a70e8/counterexamples?include_count=none&version=2017-04-21"
  }
}


In [30]:
response = conversation.update_counterexample(workspace_id=workspace_id,
                                              text='I want financial advice today.',
                                              new_text='I want financial advice for tomorrow.')
print(json.dumps(response, indent=2))

{
  "text": "I want financial advice for tomorrow.",
  "created": "2017-05-04T19:44:48.903Z",
  "updated": "2017-05-04T19:44:50.444Z"
}


In [31]:
response = conversation.delete_counterexample(workspace_id=workspace_id,
                                              text='I want financial advice for tomorrow.')
print(json.dumps(response, indent=2))

{}


## Entities

In [32]:
values = [{"value": "juice"}]

In [33]:
response = conversation.create_entity(workspace_id=workspace_id,
                                      entity='test_entity',
                                      description='A test entity.',
                                      values=values)
print(json.dumps(response, indent=2))

{
  "entity": "test_entity",
  "created": "2017-05-04T19:44:51.395Z",
  "updated": "2017-05-04T19:44:51.395Z",
  "metadata": null,
  "description": "A test entity."
}


In [34]:
response = conversation.get_entity(workspace_id=workspace_id,
                                   entity='test_entity',
                                   export=True)
print(json.dumps(response, indent=2))

{
  "entity": "test_entity",
  "values": [
    {
      "value": "juice",
      "created": "2017-05-04T19:44:51.395Z",
      "updated": "2017-05-04T19:44:51.395Z",
      "metadata": null
    }
  ],
  "created": "2017-05-04T19:44:51.395Z",
  "updated": "2017-05-04T19:44:51.395Z",
  "metadata": null,
  "description": "A test entity."
}


In [35]:
response = conversation.list_entities(workspace_id=workspace_id)
print(json.dumps(response, indent=2))

{
  "entities": [
    {
      "entity": "test_entity",
      "created": "2017-05-04T19:44:51.395Z",
      "updated": "2017-05-04T19:44:51.395Z",
      "metadata": null,
      "description": "A test entity."
    }
  ],
  "pagination": {
    "refresh_url": "/v1/workspaces/e326ef7b-bde6-4c6c-9573-aedd9e5a70e8/entities?include_count=none&version=2017-04-21&export=none"
  }
}


In [36]:
response = conversation.update_entity(workspace_id=workspace_id,
                                      entity='test_entity',
                                      new_description='An updated test entity.')
print(json.dumps(response, indent=2))

{
  "entity": "test_entity",
  "created": "2017-05-04T19:44:51.395Z",
  "updated": "2017-05-04T19:44:53.110Z",
  "metadata": null,
  "description": "An updated test entity."
}


In [37]:
response = conversation.delete_entity(workspace_id=workspace_id,
                                      entity='test_entity')
print(json.dumps(response, indent=2))

{}


## Synonyms

In [38]:
values = [{"value": "orange juice"}]
conversation.create_entity(workspace_id, 'beverage', values=values)

{'created': '2017-05-04T19:44:54.077Z',
 'description': None,
 'entity': 'beverage',
 'metadata': None,
 'updated': '2017-05-04T19:44:54.077Z'}

In [39]:
response = conversation.create_synonym(workspace_id, 'beverage', 'orange juice', 'oj')
print(json.dumps(response, indent=2))

{
  "created": "2017-05-04T19:44:54.613Z",
  "synonym": "oj",
  "updated": "2017-05-04T19:44:54.613Z"
}


In [40]:
response = conversation.get_synonym(workspace_id, 'beverage', 'orange juice', 'oj')
print(json.dumps(response, indent=2))

{
  "created": "2017-05-04T19:44:54.613Z",
  "synonym": "oj",
  "updated": "2017-05-04T19:44:54.613Z"
}


In [41]:
response = conversation.list_synonyms(workspace_id, 'beverage', 'orange juice')
print(json.dumps(response, indent=2))

{
  "synonyms": [
    {
      "created": "2017-05-04T19:44:54.613Z",
      "synonym": "oj",
      "updated": "2017-05-04T19:44:54.613Z"
    }
  ],
  "pagination": {
    "refresh_url": "/v1/workspaces/e326ef7b-bde6-4c6c-9573-aedd9e5a70e8/entities/beverage/values/orange%20juice/synonyms?include_count=none&version=2017-04-21"
  }
}


In [42]:
response = conversation.update_synonym(workspace_id, 'beverage', 'orange juice', 'oj', 'OJ')
print(json.dumps(response, indent=2))

{
  "created": "2017-05-04T19:44:54.613Z",
  "synonym": "OJ",
  "updated": "2017-05-04T19:44:56.139Z"
}


In [43]:
response = conversation.delete_synonym(workspace_id, 'beverage', 'orange juice', 'OJ')
print(json.dumps(response, indent=2))

{}


In [44]:
conversation.delete_entity(workspace_id, 'beverage')

{}

## Values

In [45]:
conversation.create_entity(workspace_id, 'test_entity')

{'created': '2017-05-04T19:44:57.676Z',
 'description': None,
 'entity': 'test_entity',
 'metadata': None,
 'updated': '2017-05-04T19:44:57.676Z'}

In [46]:
response = conversation.create_value(workspace_id, 'test_entity', 'test')
print(json.dumps(response, indent=2))

{
  "value": "test",
  "created": "2017-05-04T19:44:58.202Z",
  "updated": "2017-05-04T19:44:58.202Z",
  "metadata": null
}


In [47]:
response = conversation.get_value(workspace_id, 'test_entity', 'test')
print(json.dumps(response, indent=2))

{
  "value": "test",
  "created": "2017-05-04T19:44:58.202Z",
  "updated": "2017-05-04T19:44:58.202Z",
  "metadata": null
}


In [48]:
response = conversation.list_values(workspace_id, 'test_entity')
print(json.dumps(response, indent=2))

{
  "values": [
    {
      "value": "test",
      "created": "2017-05-04T19:44:58.202Z",
      "updated": "2017-05-04T19:44:58.202Z",
      "metadata": null
    }
  ],
  "pagination": {
    "refresh_url": "/v1/workspaces/e326ef7b-bde6-4c6c-9573-aedd9e5a70e8/entities/test_entity/values?include_count=none&version=2017-04-21&export=none"
  }
}


In [49]:
response = conversation.update_value(workspace_id, 'test_entity', 'test', 'example')
print(json.dumps(response, indent=2))

{
  "value": "example",
  "created": "2017-05-04T19:44:58.202Z",
  "updated": "2017-05-04T19:44:59.661Z",
  "metadata": null
}


In [50]:
response = conversation.delete_value(workspace_id, 'test_entity', 'example')
print(json.dumps(response, indent=2))

{}


In [51]:
conversation.delete_entity(workspace_id, 'test_entity')

{}

## Logs

In [52]:
response = conversation.list_logs(workspace_id=workspace_id)
print(json.dumps(response, indent=2))

{
  "logs": [
    {
      "request": {
        "input": {
          "text": "Can I order a pizza?"
        }
      },
      "response": {
        "intents": [],
        "entities": [],
        "input": {
          "text": "Can I order a pizza?"
        },
        "output": {
          "log_messages": [
            {
              "level": "warn",
              "msg": "No dialog node matched for the input at a root level."
            },
            {
              "level": "warn",
              "msg": "No dialog node condition matched to true in the last dialog round - context.nodes_visited is empty. Falling back to the root node in the next round."
            }
          ],
          "text": []
        },
        "context": {
          "conversation_id": "2fa01d07-f0cb-4b35-b158-d4448016bb5b",
          "system": {
            "dialog_stack": [
              {
                "dialog_node": "root"
              }
            ],
            "dialog_turn_counter": 1,
            "dialo

## Cleanup (Delete Pizza Chatbot)

Let's cleanup by deleting the pizza chatbot, since it is no longer needed.

In [53]:
# clean-up by deleting the workspace
conversation.delete_workspace(workspace_id=workspace_id)

{}